### Edgar Moises Hernandez-Gonzalez
#### 28/06/20
#### Clasificacion de calculo mental en EEG con CNN-2D + LSTM
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Escalogramas CWT (3D)
##### Resize proporcional de (45 x n_canales / 2) x 1440 / 2 [720x672]

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import time
import pywt
import cv2
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve
from keras.models import load_model

In [5]:
# todos los canales
x = pd.read_csv("/content/drive/My Drive/MentalCalculation/Datos/MeCa_car_01_3.csv", header=None)

# etiquetas
y = pd.read_csv("/content/drive/My Drive/MentalCalculation/Datos/EtiquetasMeCa01ACLL-03.csv", header=None).values

In [6]:
print(x.shape)
print(y.shape)

(304, 43008)
(304, 1)


In [7]:
# unir 32 canales y resize de tamaño proporcional
def unir_escalogramas_vertical(data):
  dim = (672, 720) #ancho, alto
  fs = 512
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  datos = np.zeros((data.shape[0],720,672))
  temporal = np.zeros((1440,1344))
  for i in range(data.shape[0]): #n muestras
    for j in range(32): #n canales
      sig = data.iloc[i, j*1344:(j+1)*1344]
      #sig = sig[160:1184]
      #escalograma CWT
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3', sampling_period = (1 / 512))
      temporal[j*45:(j+1)*45, :] = abs(coef)
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [8]:
# llamar a unir_escalogramas_vertical
inicio = time.time()

x = unir_escalogramas_vertical(x)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
Tiempo: 412.0258331298828


In [9]:
print(x.shape)

(304, 720, 672)


In [10]:
# convertir a float
x = x.astype('float32')

# escalar los valores en un rango de 0 a 1 (normalizar)
x /= np.ceil(np.max(x))

In [11]:
# convertir de 3D a 5D
x = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2], 1))

print(x.shape)

(304, 1, 720, 672, 1)


In [12]:
model = load_model('/content/drive/My Drive/MentalCalculation/Modelos/CNNRNN_CWT_01_1_CV.hdf5')

In [13]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 1, 720, 672, 8)    368       
_________________________________________________________________
time_distributed_12 (TimeDis (None, 1, 360, 336, 8)    0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 1, 360, 336, 8)    2888      
_________________________________________________________________
time_distributed_14 (TimeDis (None, 1, 180, 168, 8)    0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 1, 241920)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 8)                 7741728   
_________________________________________________________________
dense_5 (Dense)              (None, 32)               

In [14]:
test_loss, test_acc = model.evaluate(x, y, verbose=0)

In [15]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.6137
accuracy: 0.7434


In [16]:
probabilidades = model.predict(x)
y_pred_todo = np.argmax(probabilidades, 1) #1=fila

In [17]:
f1 = f1_score(y, y_pred_todo, average='macro')
print("f1:", f1)

f1: 0.715


In [18]:
matriz_confusion = confusion_matrix(y, y_pred_todo)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[ 65  60]
 [ 18 161]]


In [19]:
probabilidades = probabilidades[:, 1]
auc = roc_auc_score(y, probabilidades)
#fpr, tpr, thresholds = roc_curve(y, probabilidades)

print("AUC: %.4f" % auc)
#plt.plot(fpr, tpr)
#plt.grid()
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.show()

AUC: 0.8473
